In [6]:
!pip install transformers torch Pillow requests accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [7]:
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch
import accelerate
from PIL import Image
import requests

processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")

model = LlavaNextForConditionalGeneration.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf", torch_dtype=torch.float16, low_cpu_mem_usage=False) 
model.to("cuda:0")

# prepare image and text prompt, using the appropriate prompt template
url = "https://github.com/haotian-liu/LLaVA/blob/1a91fc274d7c35a9b50b3cb29c4247ae5837ce39/images/llava_v1_5_radar.jpg?raw=true"
image = Image.open(requests.get(url, stream=True).raw)
prompt = "[INST] <image>\nWhat is shown in this image? [/INST]"

inputs = processor(prompt, image, return_tensors="pt").to("cuda:0")

# autoregressively complete prompt
output = model.generate(**inputs, max_new_tokens=100)

print(processor.decode(output[0], skip_special_tokens=True))


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST]  
What is shown in this image? [/INST] The image appears to be a radar chart, which is a type of multi-dimensional plot that displays values for multiple quantitative variables represented on axes starting from the same point. This particular radar chart is showing the performance of different models or systems across various metrics.

The axes represent different metrics or benchmarks, such as MM-Vet, MM-Vet, MM-Vet, MM-Vet, MM-Vet, MM-


In [13]:
# prepare image and text prompt, using the appropriate prompt template
#url = "https://media.architecturaldigest.com/photos/65650bc6fff3e87c74ab695b/16:9/w_2240,c_limit/GettyImages-507971418.jpg"
url ="https://images.ctfassets.net/1aemqu6a6t65/46MJ6ER585Rwl3NraEIoGL/784c5eb5d87f576b5548b1a2255f08e7/tripadvisortimessquare_taggeryanceyiv_5912?w=1200&h=800&q=75"
image = Image.open(requests.get(url, stream=True).raw)
prompt = "[INST] <image>\nWhat is this place and what are things to do when I visit there? [/INST]"

inputs = processor(prompt, image, return_tensors="pt").to("cuda:0")

# autoregressively complete prompt
output = model.generate(**inputs, max_new_tokens=1000)

print(processor.decode(output[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST]  
What is this place and what are things to do when I visit there? [/INST] This image depicts Times Square in New York City, which is known for its bright, colorful billboards and advertisements, as well as its bustling atmosphere. Times Square is a major cultural and entertainment hub in the city, and it's a popular destination for both tourists and locals.

When visiting Times Square, you can:

1. **Enjoy the Lights**: The area is famous for its bright, neon lights that illuminate the buildings and create a vibrant, festive atmosphere.

2. **Shop**: There are numerous shops, from high-end retailers to souvenir stores, where you can find a wide range of items.

3. **See a Show**: Times Square is home to several theaters where you can catch Broadway shows.

4. **Eat**: There are many restaurants and cafes in the area, offering a variety of cuisines.

5. **People-Watch**: Times Square is a great place to people-watch and observe the hustle and bustle of city life.

6. **Visit the